<a href="https://colab.research.google.com/github/velmer/information-retrieval/blob/master/lab-07/Word2Vec_and_WMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import pandas as pd
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê\-]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Carregando Modelo Word2Vec

In [102]:
model = Word2Vec.load('pt.bin')
# Normalizing vectors
model.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Carregando Notícias

In [103]:
news = pd.read_csv('https://raw.githubusercontent.com/velmer/retrieval-information/master/lab-06/results.csv', usecols=['title'])
news.head(5)

,title
0,“A sociedade foi Rubens Paiva não os facínora...
1,Justiça suspende decisão que proibia Forças Ar...
2,Governo Bolsonaro prega “negacionismo históric...
3,Quando os pais de Gabo perceberam que tinham u...
4,Rádios canadenses banem músicas de Michael Jac...


# Pré-processamento dos títulas das notícias

In [0]:
def preprocess_text(text):
  preprocessed_text = tknz.tokenize(text)
  preprocessed_text = [word for word in preprocessed_text if word.lower() not in stopwords]
  return preprocessed_text

In [105]:
news['preprocessed_text'] = news.apply(lambda row: preprocess_text(row['title']), axis=1)
news.head(5)

,title,preprocessed_text
0,“A sociedade foi Rubens Paiva não os facínora...,"[sociedade, Rubens, Paiva, facínoras, mataram]"
1,Justiça suspende decisão que proibia Forças Ar...,"[Justiça, suspende, decisão, proibia, Forças, ..."
2,Governo Bolsonaro prega “negacionismo históric...,"[Governo, Bolsonaro, prega, negacionismo, hist..."
3,Quando os pais de Gabo perceberam que tinham u...,"[pais, Gabo, perceberam, filho, mentiroso]"
4,Rádios canadenses banem músicas de Michael Jac...,"[Rádios, canadenses, banem, músicas, Michael, ..."


# Gerando *Word Embeddings*

In [0]:
def get_word_embeddings(text):
  text_we = []
  for word in text:
    try:
      text_we.append(model[word])
    except:
      continue
  return text_we

In [107]:
news['word_embeddings'] = news.apply(lambda row: get_word_embeddings(row['preprocessed_text']), axis=1)
news.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


,title,preprocessed_text,word_embeddings
0,“A sociedade foi Rubens Paiva não os facínora...,"[sociedade, Rubens, Paiva, facínoras, mataram]","[[0.0088292025, -0.1398486, -0.08332149, 0.084..."
1,Justiça suspende decisão que proibia Forças Ar...,"[Justiça, suspende, decisão, proibia, Forças, ...","[[-0.018364985, 0.017337892, 0.02463759, 0.033..."
2,Governo Bolsonaro prega “negacionismo históric...,"[Governo, Bolsonaro, prega, negacionismo, hist...","[[0.0646153, -0.01434645, -0.076051675, -0.053..."
3,Quando os pais de Gabo perceberam que tinham u...,"[pais, Gabo, perceberam, filho, mentiroso]","[[-0.032860883, 0.03092446, -0.027608456, -0.0..."
4,Rádios canadenses banem músicas de Michael Jac...,"[Rádios, canadenses, banem, músicas, Michael, ...","[[-0.021029595, 0.06848933, 0.013900012, -0.03..."


# Cálculo do WMD

In [0]:
from math import inf
from math import sqrt

def calculate_wmd(news1, news2):
  wmd = 0
  for we1 in news1.word_embeddings:
    min_dist = inf
    for we2 in news2.word_embeddings:
      min_dist = min(min_dist, sqrt(sum((we1 - we2) ** 2)))
    wmd += (min_dist / float(len(news1.word_embeddings)))
  return wmd

In [0]:
news1 = news.iloc[0]
news2 = news.iloc[1]
assert calculate_wmd(news1, news1) == 0
assert calculate_wmd(news2, news2) == 0
assert calculate_wmd(news1, news2) != 0

# Top-3 notícias mais similares

In [0]:
def find_top3_similar_news(base_news):
  distances_to_every_other_news = []
  for index, current_news in news.iterrows():
    if base_news.title != current_news.title:
      distance = calculate_wmd(base_news, current_news)
      distances_to_every_other_news.append((index, distance))
  distances_to_every_other_news.sort(key=lambda t: t[1])
  return distances_to_every_other_news[:3]

In [133]:
base_news = news.iloc[2]
print("Notícia base:", base_news.title)
print()

assert calculate_wmd(base_news, base_news) == 0

top3_similar_news = find_top3_similar_news(base_news)

top3_similar_news_indexes = list(map(lambda n: n[0], top3_similar_news))
top3_similar_news_wmds = list(map(lambda n: n[1], top3_similar_news))
top3_similar_news_titles = list(map(lambda i: news.iloc[i].title, top3_similar_news_indexes))

top3_similar_news_data = {
    'Índice': top3_similar_news_indexes,
    'WMD': top3_similar_news_wmds,
    'Título': top3_similar_news_titles,
}

top3_similar_news_df = pd.DataFrame(data=top3_similar_news_data)
top3_similar_news_df.index += 1
pd.set_option('display.max_colwidth', -1)
top3_similar_news_df

Notícia base: Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura



,Índice,WMD,Título
1,170,0.834816,A história não pode ser apagada ou reescrita. Lembrar que houve ditadura valoriza as conquistas de hoje
2,76,0.869985,Trump reconhece soberania de Israel sobre as Colinas de Golã
3,184,0.873941,Reforma europeia garante direitos de autor na Internet mas levanta receios sobre liberdade


Os resultados foram satisfatórios, visto que a notícia que teve a menor distância é a única do Top 3 que possui uma palavra igual a da notícia base: "ditadura". Além disso, na notícia base temos a palavra "histórico" enquanto o 1º lugar possui "história", as quais são palavras com significado e escrita bem próximas.

O 2º lugar menciona Trump, presidente dos EUA, enquanto a notícia base menciona o governo Bolsonaro, presidente do Brasil, ambos os termos no papel de sujeito da frase, o que teve um efeito importante no cálcula da distância entre as duas frases.

Já o 3º lugar, como esperado, é o que menos se aproxima da notícia base. Porém, similar ao caso do 2º lugar, também possui uma entidade relacionada ao governo - "Reforma europeia" - como sujeito na frase. Além disso, possui o trecho "receios sobre liberdade", o que faz sentido quando comparamos com a notícia base, a qual menciona a palavra "ditadura".

### Comparando notícia base com uma notícia pouco similar

In [142]:
NOT_SIMILAR_INDEX = 93
not_similar_news = news.iloc[NOT_SIMILAR_INDEX]
wmd_not_similar = calculate_wmd(base_news, not_similar_news)

not_similar_news_data = {
    'Índice': [NOT_SIMILAR_INDEX],
    'WMD': [wmd_not_similar],
    'Título': [not_similar_news.title],
}

not_similar_news_df = pd.DataFrame(data=not_similar_news_data)
not_similar_news_df.index += 1
pd.set_option('display.max_colwidth', -1)
not_similar_news_df

,Índice,WMD,Título
1,93,1.441874,24 horas em Gaza


Quando comparamos a notícia base com uma notícia que possui um título bastante diferente - tamanho, contexto, elementos sintáticos - obtemos um WMD próximo do dobro para as notícias que entraram no Top-3 notícias mais similares.